## 抓取富邦人壽問答集

In [3]:
import requests
res = requests.get('https://www.fubon.com/life/direct/Faq.html')
res.encoding

'ISO-8859-1'

In [5]:
res.encoding = 'utf-8'
#res.text

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')

In [11]:
questions = [q.text for q  in soup.select('.faq__q')]
#questions

In [15]:
answers = [q.text.strip() for q  in soup.select('.faq__a')]

In [19]:
qa_ary = []
for q, a in zip(questions, answers):
    qa_ary.append({'questions':q,'answer':a})

In [21]:
import pandas
qa_df = pandas.DataFrame(qa_ary)
qa_df.to_excel('fubonqa.xlsx')

## 產生一個自動問答機器人

In [22]:
import pandas
fubonqa = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/fubonpy/master/data/fubonqa.xlsx')

In [40]:
import jieba

q = '請問要保人是什麼?'

corpus    = [' '.join(jieba.cut(q))]
questions = [q]
answers   = ['DUMMY']
for rec in fubonqa.iterrows():
    corpus.append(' '.join(jieba.cut(rec[1].questions)))
    questions.append(rec[1].questions)
    answers.append(rec[1].answer)

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X

<31x114 sparse matrix of type '<class 'numpy.int64'>'
	with 183 stored elements in Compressed Sparse Row format>

In [42]:
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(X[0], X)
pos = cs[0].argsort()[::-1][1]
questions[pos]
answers[pos]

'要保人是指對保險標的具有保險利益，向保險人申請訂立保險契約，並負有交付保險費義務之人。'

In [51]:
import jieba

q = '請問要保人是什麼?'

def autoQA(q,threshold):
    corpus    = [' '.join(jieba.cut(q))]
    questions = [q]
    answers   = ['DUMMY']
    for rec in fubonqa.iterrows():
        corpus.append(' '.join(jieba.cut(rec[1].questions)))
        questions.append(rec[1].questions)
        answers.append(rec[1].answer)
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    cs = cosine_similarity(X[0], X)
    pos = cs[0].argsort()[::-1][1]
    if cs[0][pos] > threshold:
        return answers[pos]
    else:
        return '我現在還在學習您的問題, 等我變聰明以後我就可以回答您了'

In [53]:
autoQA('請問要保人是什麼?', 0.5)

'要保人是指對保險標的具有保險利益，向保險人申請訂立保險契約，並負有交付保險費義務之人。'

In [54]:
autoQA('請問今天天氣好嗎', 0.5)

'我現在還在學習您的問題, 等我變聰明以後我就可以回答您了'

## 產生一個可對話機器人

In [58]:
import speech_recognition as sr
def listenToMe():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("請說出您的問題!")
        audio = r.listen(source)
    return r.recognize_google(audio, language='zh-TW')

In [59]:
listenToMe()

請說出您的問題!


'請問什麼是要保人'

In [60]:

from gtts import gTTS
from pygame import mixer
mixer.init()

import tempfile
def speak(sentence):
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts = gTTS(text=sentence, lang='zh')
        tts.save("{}.mp3".format(fp.name))
        mixer.music.load('{}.mp3'.format(fp.name))
        mixer.music.play()
speak('大家好')

In [67]:
speak(autoQA(listenToMe(), 0.5))

請說出您的問題!
